<a href="https://colab.research.google.com/github/robruenes/mlearned/blob/main/mlearned_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import dependencies

In [ ]:
from google.colab import drive
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np

# Authorize drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# Read in full dataset

In [ ]:
training_set_path = '/content/drive/My Drive/mlearned/training_set.csv'
df = pd.read_csv(training_set_path, sep='\t', encoding='utf-8')
# Brand new players haven't answered questions in every category yet,
# leading to some nulls in the dataset.
df = df.dropna()

X_all_cols = df.drop(['Result'], axis=1)
X_fewer_cols = X_all_cols.drop([
    'Wins', 'Losses', 'Ties', 'Points in Standings',
    'Match Points Differential', 'Total Match Points','Total Correct Answers',
    'Total Points Allowed', 'Correct Answers Allowed','Unforced Points Allowed',
    'Defensive Efficiency', 'Wins by Forfeit', 'Losses by Forfeit',
    '3 point questions answered correctly'],
    axis=1)
y = df["Result"]

# Create training and test sets.
Xa_train, Xa_test, ya_train, ya_test = train_test_split(X_all_cols, y, test_size=0.3, random_state=42)

Xs_train, Xs_test, ys_train, ys_test = train_test_split(X_fewer_cols, y, test_size=0.3, random_state=42)

# Train models

In [ ]:
model_s = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(3, activation='linear')])

model_s.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

model_s.fit(Xs_train, ys_train, epochs=100, validation_split=0.2)

Epoch 1/100
300/300 [==============================] - 2s 3ms/step - loss: 1.0222 - val_loss: 1.0128
Epoch 2/100
300/300 [==============================] - 1s 2ms/step - loss: 1.0029 - val_loss: 1.0095
Epoch 3/100
300/300 [==============================] - 1s 2ms/step - loss: 0.9985 - val_loss: 1.0081
Epoch 4/100
300/300 [==============================] - 1s 2ms/step - loss: 0.9952 - val_loss: 1.0057
Epoch 5/100
300/300 [==============================] - 1s 2ms/step - loss: 0.9938 - val_loss: 1.0067
Epoch 6/100
300/300 [==============================] - 1s 3ms/step - loss: 0.9918 - val_loss: 1.0082
Epoch 7/100
300/300 [==============================] - 1s 3ms/step - loss: 0.9904 - val_loss: 1.0043
Epoch 8/100
300/300 [==============================] - 1s 3ms/step - loss: 0.9885 - val_loss: 1.0054
Epoch 9/100
300/300 [==============================] - 1s 3ms/step - loss: 0.9857 - val_loss: 1.0035
Epoch 10/100
300/300 [==============================] - 1s 3ms/step - loss: 0.9850 - val_lo

In [ ]:
model_a = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(3, activation='linear')])

model_a.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

model_a.fit(Xa_train, ya_train, epochs=100, validation_split=0.2)

Epoch 1/100
300/300 [==============================] - 2s 5ms/step - loss: 4.1389 - val_loss: 4.3001
Epoch 2/100
300/300 [==============================] - 1s 3ms/step - loss: 3.0283 - val_loss: 3.5242
Epoch 3/100
300/300 [==============================] - 1s 2ms/step - loss: 2.4708 - val_loss: 1.5342
Epoch 4/100
300/300 [==============================] - 1s 2ms/step - loss: 2.4398 - val_loss: 3.3735
Epoch 5/100
300/300 [==============================] - 1s 2ms/step - loss: 2.1740 - val_loss: 4.4323
Epoch 6/100
300/300 [==============================] - 1s 2ms/step - loss: 2.0558 - val_loss: 2.3299
Epoch 7/100
300/300 [==============================] - 1s 2ms/step - loss: 1.7559 - val_loss: 1.9236
Epoch 8/100
300/300 [==============================] - 1s 2ms/step - loss: 1.9844 - val_loss: 3.8119
Epoch 9/100
300/300 [==============================] - 1s 2ms/step - loss: 2.1655 - val_loss: 1.8140
Epoch 10/100
300/300 [==============================] - 1s 2ms/step - loss: 1.7610 - val_lo

# Try the models out

In [ ]:
Xa_test = Xa_test.reset_index(drop=True)
ya_test = ya_test.reset_index(drop=True)

predicted_logits_a = model_a.predict(Xa_test)
# Apply softmax and get the predicted category
probabilities_a = tf.nn.softmax(predicted_logits_a)
predicted_categories_a = tf.argmax(probabilities_a, axis=1)
matches_a = 0.0
total_entries_a = len(ya_test)
for i in range(0, total_entries_a):
  if ya_test[i] == predicted_categories_a[i]:
    matches_a = matches_a + 1.0

print("Accuracy: ", matches_a/float(total_entries_a))


161/161 [==============================] - 0s 2ms/step
Accuracy:  0.4660042859925969


In [ ]:
Xs_test = Xs_test.reset_index(drop=True)
ys_test = ys_test.reset_index(drop=True)

predicted_logits_s = model_s.predict(Xs_test)
# Apply softmax and get the predicted category
probabilities_s = tf.nn.softmax(predicted_logits_s)
predicted_categories_s = tf.argmax(probabilities_s, axis=1)
matches_s = 0.0
total_entries_s = len(ys_test)
for i in range(0, total_entries_s):
  if ys_test[i] == predicted_categories_s[i]:
    matches_s = matches_s + 1.0

print("Accuracy: ", matches_s/float(total_entries_s))


161/161 [==============================] - 0s 2ms/step
Accuracy:  0.46697837521917007


In [ ]:
# Save models.
model_a.save('/content/drive/My Drive/mlearned/model_a.keras')
model_s.save('/content/drive/My Drive/mlearned/model_s.keras')